In [4]:
import numpy as np
import cv2
from PIL import Image as im
import cv2
import utlis

ModuleNotFoundError: No module named 'utlis'

In [5]:
pathImage = "data\omr_line.v2i.yolov7pytorch\train\images\310377047_465921045601836_3357657464690243685_n_jpg.rf.32b8bfbd56bde205a8bdec2fa9460b1a.jpg"
img = cv2.imread(pathImage)
img = cv2.resize(img, (640, 640))
imgBlank = np.zeros((640,640, 3), np.uint8)
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
thres=utlis.valTrackbars()
imgThreshold = cv2.Canny(imgBlur,thres[0],thres[1])

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
